### About
  This notebook is based on /generate_module.ipynb that created by Jae Won Kim.

### Change Log
* In `create_revolute_joint()`(originally `revolute_joint()`):
  1. Assign the type of the connector of the proximal body of the `base_rev_joint` as `base` to correctly determine the base of the robot and replace `base()` function.
  2. Keep the original value of the transformation of `parent2joint`, and set the value of the transformation(rotation and translation) of `joint2child` to [0,0,0] to eliminate the minor incline in previous configuration.
  3. Temporarily comment the `r_p`, `p_p`, `r_d`, `p_d`, they seems not being used.
* In `read_rod_trans()`:
  1. Add rod configuration, specifically the rotation and translation of the connectors, between different joints

In [2]:
from datetime import date
author = "Jonas Li, Jae Won Kim"
email = "liyunzhe.jonas@berkeley.edu"
affiliation = "UC Berkeley"
def generate_header(header_id, header_name):
    return ModuleHeader(ID=header_id,
                        name=header_name,
                        date=date.today().strftime('%Y-%m-%d'),
                        author=author,
                        email=email,
                        affiliation=affiliation
                        )

In [41]:
from util import *
from timor.Geometry import Mesh
import os
import pygad
from timor.Bodies import Body, Connector, Gender
from timor.utilities.transformation import Transformation
from timor.utilities.spatial import rotX, rotY, rotZ
from timor.Module import AtomicModule, ModulesDB, ModuleHeader
from timor import ModuleAssembly, ModulesDB
from timor.configuration_search.GA import GA
from timor.Joints import Joint
from timor.Geometry import Box, ComposedGeometry, Cylinder, Sphere, Mesh
from timor.Module import *
from timor.utilities.dtypes import Lexicographic

Steps:
1. Get proximal/distal body geometry, inertial, and origin
2. Define connector of proximal/distal body
3. Define proximal/distal body
4. Define joint
   * Define transformation between joint and connector 

In [36]:
diameter = 80 / 1000
ROT_X = Transformation.from_rotation(rotX(np.pi)[:3, :3]) 
def eef() -> ModulesDB:
    """Creates a simplified end effector module."""
    geometry = Sphere({'r': diameter / 5}, pose=Transformation.from_translation([0, 0, diameter / 2]))
    c_robot = Connector('robot2eef', ROT_X, gender=Gender.h, connector_type='default', size=[25 / 1000, 25 / 1000])
    c_world = Connector('end-effector', gender=Gender.m, connector_type='eef', size=[25 / 1000, 25 / 1000],
                        body2connector=Transformation.from_translation([0, 0, diameter / 2]))
    return ModulesDB({
        AtomicModule(generate_header("eef", "End Effector: eef"),
                     [Body('EEF', collision=geometry, connectors=[c_robot, c_world])])
    })

def create_revolute_joint(urdf_path: str, eef=None):
    dir_name = urdf_path.split('/')[1]
    urdf_dict = urdf_to_dict(urdf_path)
    joint = urdf_dict['robot']['joint']
    proximal_name = joint['parent']['link']
    distal_name = joint['child']['link']
    links = urdf_dict['robot']['link']

    
    for link in links:
        link_name = link['name']
        stl_path = link['collision']['geometry']['mesh']['filename']
        assets_path = os.path.join("assets", dir_name, dir_name, stl_path.split('/')[1], stl_path.split('/')[2])
        if link_name == proximal_name:
            proximal_inertial = link['inertial']
            proximal_origin = link['collision']['origin']
            proximal_geometry = Mesh({"file": assets_path})
        elif link_name == distal_name:
            distal_inertial = link['inertial']
            distal_origin = link['collision']['origin']
            distal_geometry = Mesh({"file": assets_path})
    
    

    diameter = 25 / 1000
    # r_p, p_p = body2connector_helper([float(x) for x in joint['origin']['xyz'].split(" ")], [float(x) for x in joint['origin']['rpy'].split(" ")], [float(x) for x in proximal_origin['xyz'].split(" ")], [float(x) for x in proximal_origin['rpy'].split(" ")])
    # r_d, p_d = body2connector_helper([float(x) for x in joint['origin']['xyz'].split(" ")], [float(x) for x in joint['origin']['rpy'].split(" ")], [float(x) for x in distal_origin['xyz'].split(" ")], [float(x) for x in distal_origin['rpy'].split(" ")])
    
    # ROT_X = Transformation.from_rotation(-rotX(np.pi/2)[:3, :3])
    # ROT_Y = Transformation.from_rotation(rotY(-np.pi)[:3, :3])
    c_type = 'base' if  joint['name'] == 'base_rev_joint' else 'default'
        
    proximal_connector = Connector(
                                    connector_id=proximal_name+"connector",
                                    body2connector=Transformation.from_roto_translation(
                                                    R=rpy_to_rotation_matrix(np.array([0,0, 0])),       
                                                    # R=r_p,
                                                    p=np.array([0, -0.0, 0]),
                                                    # p=p_p
                                    ),
                                    gender=Gender.m if not eef else Gender.h,
                                    connector_type=c_type,
                                    size=[diameter, diameter]
        )
    distal_connector = Connector(
                                    connector_id=distal_name+"connector",
                                    body2connector=Transformation.from_roto_translation(
                                                    R=rpy_to_rotation_matrix(np.array([0,0, 0])),
                                                    p=np.array([0, -0.0, 0]),            
                                                    # R=r_d,
                                                    # p=p_d
                                    ),
                                    gender=Gender.f,
                                    connector_type='default',
                                    size=[diameter, diameter]
                                    
        )            
    
    proximal = Body(proximal_name, collision=proximal_geometry,
                    connectors=[proximal_connector],
                    inertia=create_inertia(proximal_inertial)
                    )
    distal = Body(distal_name, collision=distal_geometry,
                    connectors=[distal_connector],
                    inertia=create_inertia(distal_inertial)
                    )

    r_joint = Joint(
        joint_id=joint['name'],
        joint_type=joint['type'],
        parent_body=proximal,
        child_body=distal,
        q_limits=np.array([-np.pi, np.pi]),
        torque_limit=1000,
        acceleration_limit=5,
        velocity_limit=10,
        parent2joint=Transformation.from_roto_translation(
                                                    R=rpy_to_rotation_matrix([float(x) for x in joint['origin']['rpy'].split(" ")]),
                                                    p=joint['origin']['xyz'].split(" ")
                                ),
        joint2child=Transformation.from_roto_translation(
                                                    R=rpy_to_rotation_matrix(np.array([0, 0, 0])),
                                                    p=np.array([0, -0.0, 0])
                                )
    )
    
    return AtomicModule(generate_header(joint['name'], 'Revolute Joint: ' + joint['name']), [proximal, distal], [r_joint])
    return ModulesDB({
            AtomicModule(generate_header(joint['name'], 'Revolute Joint: ' + joint['name']), [proximal, distal], [r_joint])
            # AtomicModule(generate_header(joint['name'], "R"), [proximal])
        })

In [6]:
def create_connectors_for_link(length, diameter, trans):
    connectors = []
    for i in range(2):
        connectors.append(Connector(
            f'{int(diameter * 100)}-{i}', 
            trans[i],
            gender=Gender.f if i == 0 else Gender.m,
            connector_type='default', 
            size=[diameter, diameter]))
    return connectors

def create_i_links(rod_name) -> ModulesDB:
    """For every size, creates an I-shaped link (aka a cylinder) with two connectors."""
    ROT_X = Transformation.from_rotation(rotX(-np.pi/2)[:3, :3])
    sizes = (150 / 1000, 300 / 1000, 450 / 1000)
    diameter = 25 / 1000
    links = ModulesDB()
    
    
    for size in sizes:
        rod_id = f'{rod_name}-{size}'
        trans = read_rod_trans(rod_name, size, diameter)
        header = generate_header(rod_id, rod_id)
        # connectors = (
        #     Connector(rod_id,
        #             ROT_X @ Transformation.from_translation([0, size/2, 0]) if i == 0
        #             else Transformation.from_translation([0, 0, 0.077]),
        #             gender=Gender.f if i == 0 else Gender.m,
        #             connector_type='default',
        #             size=[diameter, diameter])
        #     for i in range(2)
        # )
        # print(type(connectors))
        connectors = create_connectors_for_link(size, diameter, trans)
        geometry = Box({'x': diameter, 'y': diameter, 'z': size}, pose=Transformation.from_translation([0, 0, 0]))
        body = Body(rod_id, collision=geometry, connectors=connectors,
                    inertia=square_rod_inertia(size, diameter))
        links.add(AtomicModule(header, [body]))
    return links
    
def read_rod_trans(rod_name, length, diameter):
    if rod_name == "baseto4310":
        ROT_X = Transformation.from_rotation(rotX(-np.pi/2)[:3, :3])
        return [ROT_X @ Transformation.from_translation([0, length/2, 0]), Transformation.from_translation([0, 0, 0.077])]
    elif rod_name == "r4310to4305":
        ROT_X_90 = Transformation.from_rotation(rotX(-np.pi/2)[:3, :3])
        ROT_Z_180 = Transformation.from_rotation(rotZ(np.pi)[:3, :3])
        ROT_Y = Transformation.from_rotation(rotX(np.pi/2)[:3, :3])
        config_1 = [Transformation.from_translation([0, 0, 0]), Transformation.from_translation([0, -diameter, -length/2+diameter/2])]
        config_2 = [ROT_Y @ Transformation.from_translation([0, length/2, 0]), ROT_Y @ Transformation.from_translation([0, -length/2+diameter/2, 0])]
        return config_2
    elif rod_name == "r4310to4310":
        ROT_X_90 = Transformation.from_rotation(rotX(-np.pi/2)[:3, :3])
        ROT_Z_180 = Transformation.from_rotation(rotZ(np.pi)[:3, :3])
        ROT_Z_90 = Transformation.from_rotation(rotZ(np.pi/2)[:3, :3])
        ROT_Z_N90 = Transformation.from_rotation(rotZ(-np.pi/2)[:3, :3])
        ROT_Y = Transformation.from_rotation(rotX(np.pi/2)[:3, :3])
        ROT_Y_180 = Transformation.from_rotation(rotX(np.pi)[:3, :3])
        config_1 = [ROT_Z_180 @ ROT_Y @ Transformation.from_translation([0, length/2-diameter/2, 0]), Transformation.from_translation([0, -diameter, -length/2+diameter/2])]
        config_2 = [ROT_Z_90 @ ROT_X_90 @ ROT_Y_180 @ Transformation.from_translation([0, length/2-diameter/2, 0]), Transformation.from_translation([0, -diameter, -length/2+diameter/2])]
        return config_1

In [37]:
# Base and joint
r_4310_base = create_revolute_joint("assets/Assem_4310_BASE/Assem_4310_BASE/urdf/Assem_4310_BASE.urdf")
r_4305_joint = create_revolute_joint("assets/Assem_4305_JOINT/Assem_4305_JOINT/urdf/Assem_4305_JOINT.urdf", eef=True)
r_4310_joint = create_revolute_joint("assets/Assem_4310_JOINT/Assem_4310_JOINT/urdf/Assem_4310_JOINT.urdf")

# Links
baseto4310_links = create_i_links(rod_name="baseto4310")
r4310to4305_links = create_i_links(rod_name="r4310to4305")
r4310to4310_links = create_i_links(rod_name="r4310to4310")

eef = eef()

# Create database
db = ModulesDB()
db.add(r_4310_base)
db.add(r_4310_joint)
db.add(r_4305_joint)
db = db.union(baseto4310_links)
db = db.union(r4310to4305_links)
db = db.union(r4310to4310_links)
db = db.union(eef)
viz = db.debug_visualization()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7005/static/


In [8]:
modules = ('base_rev_joint', 'baseto4310-0.15','motor4310_rev_joint', 'r4310to4310-0.15', 'motor4310_rev_joint', 'r4310to4310-0.15', 'motor4310_rev_joint', 'r4310to4305-0.15', 'motor4305_rev_joint')
B = ModuleAssembly.from_serial_modules(db, modules)
robot = B.to_pin_robot()
viz = robot.visualize()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [18]:
our_hyperparameters = {
    'population_size': 10,
    'num_generations': 50,
    'num_genes': 6,
    'save_solutions_dir': None
}

def fitness_scalar(assembly: ModuleAssembly, ga_instance: pygad.GA, index: int) -> float:
    """We start with a very simple fitness function that returns the negative mass of the assembly"""
    if assembly.nJoints != 3:
        return -10000
    return -assembly.mass
print(db.by_id)

{'motor4310_rev_joint': <timor.Module.AtomicModule object at 0x778b6a1e3be0>, 'r4310to4305-0.15': <timor.Module.AtomicModule object at 0x778af77bc820>, 'motor4305_rev_joint': <timor.Module.AtomicModule object at 0x778af7770b20>, 'r4310to4305-0.45': <timor.Module.AtomicModule object at 0x778af77bcfa0>, 'baseto4310-0.15': <timor.Module.AtomicModule object at 0x778af77bf850>, 'baseto4310-0.3': <timor.Module.AtomicModule object at 0x778af77bdf00>, 'base_rev_joint': <timor.Module.AtomicModule object at 0x778af7770820>, 'r4310to4305-0.3': <timor.Module.AtomicModule object at 0x778af77bc970>, 'r4310to4310-0.45': <timor.Module.AtomicModule object at 0x778af77bec20>, 'r4310to4310-0.15': <timor.Module.AtomicModule object at 0x778af77bdcf0>, 'r4310to4310-0.3': <timor.Module.AtomicModule object at 0x778af77bead0>, 'baseto4310-0.45': <timor.Module.AtomicModule object at 0x778af77be200>}


In [42]:
# for connector in db.all_connectors:
#     print(connector.size)
db2 = ModulesDB.from_name('geometric_primitive_modules')
#db.connectivity_graph

In [44]:

ga = GA(db, our_hyperparameters) 
ga_optimizer = ga.optimize(fitness_function=fitness_scalar, save_best_solutions=False)

2025-02-11 00:29:54,285 Timor INFO Hyperparameters used: {"population_size": 10, "num_generations": 50, "num_genes": 6, "mutation_probability": 0.2, "num_parents_mating": 5, "keep_parents": 4, "keep_elitism": 5, "save_solutions_dir": null}
2025-02-11 00:29:54,287 Timor INFO Progress unit: ProgressUnit.GENERATIONS
Generations:   0%|          | 0/51 [00:00<?, ?it/s]2025-02-11 00:29:54,312 Timor ERROR There is no unique way to add Module Revolute Joint: motor4305_rev_joint, ID: motor4305_rev_joint to the assembly.
Traceback (most recent call last):
  File "/home/mscsim/anaconda3/envs/timor-env/lib/python3.10/site-packages/pygad/pygad.py", line 1697, in cal_pop_fitness
    fitness = self.fitness_func(self, sol, sol_idx)
  File "/home/mscsim/anaconda3/envs/timor-env/lib/python3.10/site-packages/timor/configuration_search/GA.py", line 381, in wrapper
    result = fitness_func(_ga, individual, individual_idx)
  File "/home/mscsim/anaconda3/envs/timor-env/lib/python3.10/site-packages/timor/con

InvalidAssemblyError: There is no unique way to add Module Revolute Joint: motor4305_rev_joint, ID: motor4305_rev_joint to the assembly.

# -------------------------------------------------------

# Should not use

In [30]:
urdf_path = "assets/Assem_4310_BASE/Assem_4310_BASE/urdf/Assem_4310_BASE.urdf"
dir_name = urdf_path.split('/')[1]
urdf_dict = urdf_to_dict(urdf_path)
urdf_dict

{'robot': {'name': 'Assem_4310_BASE',
  'link': [{'name': 'base_motor_link',
    'inertial': {'origin': {'xyz': '1.6355E-05 2.1044E-06 0.030945',
      'rpy': '0 0 0'},
     'mass': {'value': '1.1937'},
     'inertia': {'ixx': '0.0011798',
      'ixy': '-3.4788E-08',
      'ixz': '2.8837E-07',
      'iyy': '0.0010863',
      'iyz': '5.0519E-08',
      'izz': '0.0016066'}},
    'visual': {'origin': {'xyz': '0 0 0', 'rpy': '0 0 0'},
     'geometry': {'mesh': {'filename': '../meshes/base_motor_link.STL'}},
     'material': {'name': '', 'color': {'rgba': '0.89804 0.91765 0.92941 1'}}},
    'collision': {'origin': {'xyz': '0 0 0', 'rpy': '0 0 0'},
     'geometry': {'mesh': {'filename': '../meshes/base_motor_link.STL'}}}},
   {'name': 'base_out_link',
    'inertial': {'origin': {'xyz': '2.3272E-17 0.0011054 0.008691',
      'rpy': '0 0 0'},
     'mass': {'value': '0.088545'},
     'inertia': {'ixx': '2.4599E-05',
      'ixy': '-1.5532E-21',
      'ixz': '3.7828E-22',
      'iyy': '2.5766E-05

In [ ]:
def create_base(urdf_path: str) -> ModulesDB:    
    dir_name = urdf_path.split('/')[1]
    urdf_dict = urdf_to_dict(urdf_path)
    
    joint = urdf_dict['robot']['joint']
    
    # Body Geometry
    stl_path = urdf_dict['robot']['link'][0]['collision']['geometry']['mesh']['filename']
    assets_path = os.path.join("assets", dir_name, dir_name, stl_path.split('/')[1], stl_path.split('/')[2])
    stl_geometry = Mesh({"file": assets_path})

    # Connector:
    # c_world: connect with the world coordinate
    # c_robot: connect with the robot coordinate
    diameter = 25 / 1000
    ROT_X = Transformation.from_rotation(rotX(-np.pi/2)[:3, :3])
    c_world = Connector('base', ROT_X, gender=Gender.f, connector_type='base')
    c_robot = Connector('base2robot', gender=Gender.m, connector_type='default', size=[diameter, diameter])

    return AtomicModule(
                            generate_header('base', 'Base'),
                            [Body('base', 
                                  collision=stl_geometry, 
                                  connectors=[c_world, c_robot])])
    
base = create_base("assets/Assem_4310_BASE/Assem_4310_BASE/urdf/Assem_4310_BASE.urdf")
base.debug_visualization()